In [ ]:
sharedinputpath = snakemake.params["sharedinputpath"]
desired_regions = snakemake.params.aggregated_regions
weatherdata = snakemake.input.weatherdata

cf_file = snakemake.output.cf_file

In [ ]:
onshore_turbine = "Vestas_V112_3MW"
offshore_bottom_turbine = "oedb:V164"
offshore_floating_turbine = "oedb:V164"

panel = "CSi"
orientation = "latitude_optimal"

In [ ]:
geodata_files = {
    "onshore": snakemake.input.euroshape,
    "offshore_bottom": snakemake.input.eurooffshoreshape,
}

In [ ]:
import logging

import atlite

logging.basicConfig(level=logging.INFO)

import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np

In [ ]:
%psource atlite.Cutout.convert_and_aggregate

In [ ]:
boundaries = []
for geodata_file_name, geodata_file_path in geodata_files.items():
    print(geodata_file_path)
    boundaries.append(gpd.read_file(geodata_file_path))

boundaries = pd.concat(boundaries).bounds

In [ ]:
boundaries = boundaries.groupby(lambda x: "bountry").agg(
    {"minx": "min", "miny": "min", "maxx": "max", "maxy": "max"}
)

# boundaries["minx"] = boundaries["minx"] - 2
# boundaries["miny"] = boundaries["miny"] - 2
# boundaries["maxx"] = boundaries["maxx"] + 2
# boundaries["maxy"] = boundaries["maxy"] + 2

In [ ]:
ds = xr.open_dataset(weatherdata, chunks="auto")

In [ ]:
cutout = atlite.Cutout(
    path="../3_intermediate_data/intermediatecutout.nc",
    data=ds.sel(
        x=slice(
            boundaries.loc["bountry", "minx"],
            boundaries.loc["bountry", "maxx"],
        ),
        y=slice(
            boundaries.loc["bountry", "miny"],
            boundaries.loc["bountry", "maxy"],
        ),
    ),
)

cutout.prepare()

In [ ]:
# Wind bias-correction - 100m wind speed

In [ ]:
if snakemake.params.bias_correction:
    # Path to bias correction ratios
    windbiaspath = snakemake.input.biaswinddata

    # Get wind speed at 100m from cutout
    wnd_100m = cutout.data["wnd100m"]
    attrs = wnd_100m.attrs

    # Load dataset containing bias correction ratios
    gwa2_ratio = xr.open_dataset(windbiaspath, chunks="auto")

    # Select bias correction ratio for wind speed at 100m and
    # rename coordinates to the same names in cutout
    gwa2_ratio_100m = (
        gwa2_ratio
        # .sel(height=100, drop=True)
        # .drop_vars("spatial_ref")
        .rename({"longitude": "x", "latitude": "y"})
    )

    # Actual bias-correction ratios (from GWA2) has finner resolution
    # than ERA5 data (bias-correction ratios: 0.025, and ERA5: 0.25)
    # To match both data, we aggregate (average) ratios to ERA5 grid cells.
    # Also, the coordinates in both datasets do not match. So, we
    # interpolated according to ERA5 grid cells.
    # TODO: In the future, we can downscale ERA5 wind speed data to GWA2
    # resolution, exclude certain grid cells to make a more sophisticated
    # filter, and then upscale to the original ERA5 resolution.

    # For now, the code aggregates the GWA2 ratio to the ERA5 resolution
    # (from 0.025 to 0.25), interpolates to ERA5 grid cells, and then
    # corrects the wind speed.

    # Calculate number of points to aggregate

    # steps dx and dy (GWA2)
    dx_gwa2 = 0.025
    dy_gwa2 = 0.025
    # ERA5
    dx_era5 = 0.25
    dy_era5 = 0.25

    # Another way
    # dx_gwa2 = np.round(gwa2_ratio_100m.coords['x'].diff('x').values[0],4)
    # dy_gwa2 = np.round(gwa2_ratio_100m.coords['y'].diff('y').values[0],4)

    # dx_era5 = np.round(wnd_100m.coords['x'].diff('x').values[0],4)
    # dy_era5 = np.round(wnd_100m.coords['y'].diff('y').values[0],4)

    # Calculate the number of points to aggregate when the data is coarsened
    x_window = int(dx_era5 / dx_gwa2)
    y_window = int(dy_era5 / dy_gwa2)

    # Coarsen and interpolate gwa2 ratios resolution (0.025)
    # to the ERA5 resolution (0.25) by averaging
    gwa2_coarsened = (
        gwa2_ratio_100m.coarsen(x=x_window, y=y_window, boundary="pad")
        .mean()
        .interp_like(wnd_100m)
    )

    # Correct 100m wind speed
    cutout.data["wnd100m"] = (
        wnd_100m * gwa2_coarsened["ratio_gwa2_era5_mean_WS"]
    ).assign_attrs(attrs)  # copy also the attributes

In [ ]:
# Solar CF

In [ ]:
%psource atlite.Cutout.convert_and_aggregate

In [ ]:
cf_solar = cutout.pv(
    panel=panel,
    orientation=orientation,
    capacity_factor_timeseries=True,
).astype(np.float32)

In [ ]:
# Onshore wind CF

In [ ]:
cf_windon = cutout.wind(
    turbine=onshore_turbine, capacity_factor_timeseries=True
).astype(np.float32)

In [ ]:
# Offshore wind CF

In [ ]:
cf_windoff_bottom = cutout.wind(
    turbine=offshore_bottom_turbine, capacity_factor_timeseries=True
).astype(np.float32)

In [ ]:
cf = xr.concat(
    [cf_solar, cf_windon, cf_windoff_bottom],
    pd.Index(["Solar", "Windonshore", "Windoffshore"], name="Tech"),
)

In [ ]:
cf.to_netcdf(
    cf_file,
    encoding={
        "capacity factor": {
            "dtype": "int16",
            "scale_factor": 0.001,
            "_FillValue": -99,
            "zlib": True,
            "complevel": 1,
        }
    },
)